In [1]:
import os
import sys
sys.path.append("../cnn")
sys.path.append("../utils")

import torch
from torch.autograd import Variable

from ipywidgets import interact, fixed
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# import seaborn as sns; sns.set()

from cnn_md import CNNMultidecoder, CNNVariationalMultidecoder
from cnn_md import CNNAdversarialMultidecoder
from cnn_md import CNNGANMultidecoder
from hao_data import HaoEvalDataset

In [2]:
# Set up environment variables for the model we want to examine
# NOT necessarily the current environment variables!!

feat_dim=80
left_context=5
right_context=5

optimizer="Adam"
learning_rate="0.0001" # Use string instead of float to prevent Python's auto-formatting...
epochs=25
batch_size=256

enc_channels=[256, 256]
enc_kernels=[3, 3]
enc_pools=[3, 3]
enc_fc=[]

latent_dim=256

dec_fc=[]
dec_channels=[256, 256]
dec_kernels=[3, 3]
dec_pools=[3, 3]

decoder_classes=["ihm", "sdm1"]

use_batch_norm=False
use_batch_norm_str = "true" if use_batch_norm else "false"
activation="ReLU"
weight_init="xavier_uniform"

enc_channels_delim="_" + "_".join(map(str, enc_channels))
if len(enc_channels) == 0:
     # need to recreate bash join behavior with empty array
    enc_channels_delim="_"
enc_kernels_delim="_" + "_".join(map(str, enc_kernels))
if len(enc_kernels) == 0:
     # need to recreate bash join behavior with empty array
    enc_kernels_delim="_"
enc_pools_delim="_" + "_".join(map(str, enc_pools))
if len(enc_pools) == 0:
     # need to recreate bash join behavior with empty array
    enc_pools_delim="_"
enc_fc_delim="_" + "_".join(map(str, enc_fc))
if len(enc_fc) == 0:
     # need to recreate bash join behavior with empty array
    enc_fc_delim="_"
    
dec_fc_delim="_" + "_".join(map(str, dec_fc))
if len(dec_fc) == 0:
     # need to recreate bash join behavior with empty array
    dec_fc_delim="_"
dec_channels_delim="_" + "_".join(map(str, dec_channels))
if len(dec_channels) == 0:
     # need to recreate bash join behavior with empty array
    dec_channels_delim="_"
dec_kernels_delim="_" + "_".join(map(str, dec_kernels))
if len(dec_kernels) == 0:
     # need to recreate bash join behavior with empty array
    dec_kernels_delim="_"
dec_pools_delim="_" + "_".join(map(str, dec_pools))
if len(dec_pools) == 0:
     # need to recreate bash join behavior with empty array
    dec_pools_delim="_"

debug_model = False
debug_str = "true" if debug_model else "false"
cnn_name = "ENC_C%s_K%s_P%s_F%s/LATENT_%d/DEC_F%s_C%s_K%s_P%s/ACT_%s_BN_%s_WEIGHT_INIT_%s/OPT_%s_LR_%s_EPOCHS_%d_BATCH_%d_DEBUG_%s" % (enc_channels_delim,
                                                                                                                                        enc_kernels_delim, 
                                                                                                                                        enc_pools_delim, 
                                                                                                                                        enc_fc_delim,
                                                                                                                                        latent_dim,
                                                                                                                                        dec_fc_delim,
                                                                                                                                        dec_channels_delim,
                                                                                                                                        dec_kernels_delim,                                                                                                                   dec_pools_delim,
                                                                                                                                        activation,
                                                                                                                                        use_batch_norm_str,
                                                                                                                                        weight_init,
                                                                                                                                        optimizer,
                                                                                                                                        learning_rate,
                                                                                                                                        epochs,
                                                                                                                                        batch_size,
                                                                                                                                        debug_str)

time_dim = (left_context + right_context + 1)
freq_dim = feat_dim

noise_ratio=0.25
print("Noise ratio %s" % str(noise_ratio))

print("Using CNN name %s" % cnn_name)

dataset = "ami-0.1"

adv_fc = [256]
adv_fc_delim="_" + "_".join(map(str, adv_fc))
if len(adv_fc) == 0:
     # need to recreate bash join behavior with empty array
    adv_fc_delim="_"
adv_activation = "Sigmoid"

gan_fc = [256]
gan_fc_delim="_" + "_".join(map(str, gan_fc))
if len(adv_fc) == 0:
     # need to recreate bash join behavior with empty array
    gan_fc_delim="_"
gan_activation = "Sigmoid"

Noise ratio 0.25
Using CNN name ENC_C_256_256_K_3_3_P_3_3_F_/LATENT_256/DEC_F__C_256_256_K_3_3_P_3_3/ACT_ReLU_BN_false_WEIGHT_INIT_xavier_uniform/OPT_Adam_LR_0.0001_EPOCHS_25_BATCH_256_DEBUG_false


In [3]:
# Set up datasets + error arrays for IHM, SDM1 baselines (dev set only)
dataset_name = "ami-0.1"

feat_dir = "/data/sls/r/u/atitus5/meng/%s" % dataset_name
am_logs = "/data/sls/r/u/atitus5/meng/am/logs/%s" % dataset_name

baseline_feats = dict()
baseline_errs = {decoder_class: dict() for decoder_class in decoder_classes}
for decoder_class in decoder_classes:
    baseline_feats[decoder_class] = HaoEvalDataset(os.path.join(feat_dir,
                                                                "%s-dev-norm.blogmel.scp" % decoder_class))
    
    expt_name = "train_%s/baseline/frame-tdnn-450x7-step0.05" % decoder_class
    err_dir = os.path.join(am_logs, expt_name)
    for predict_domain in decoder_classes:
        baseline_errs[decoder_class][predict_domain] = HaoEvalDataset(os.path.join(err_dir, "predict_%s/errors.scp" % predict_domain))

print("Set up baseline dev datasets")

# Load in error arrays for IHM->IHM, IHM->SDM1, SDM1->SDM1, SDM1->IHM (dev set only)
model_type = "ae"
adversarial = False
gan = False

augmented_errs = {decoder_class: {decoder_class_2: dict() for decoder_class_2 in decoder_classes} for decoder_class in decoder_classes}
for source_domain in decoder_classes:
    for target_domain in decoder_classes:
        expt_name = "train_%s/augmented_src_%s/frame-tdnn-450x7-step0.05/%s" % (target_domain,
                                                                                source_domain,
                                                                                cnn_name)
        err_base_dir = os.path.join(am_logs, expt_name)
        
        if adversarial:
            err_dir = os.path.join(err_base_dir, "adversarial_fc_%s_act_%s_%s_ratio%s" % (adv_fc_delim,
                                                                                          adv_activation,
                                                                                          model_type,
                                                                                          str(noise_ratio)))
        elif gan:
            err_dir = os.path.join(err_base_dir, "gan_fc_%s_act_%s_%s_ratio%s" % (gan_fc_delim,
                                                                                  gan_activation,
                                                                                  model_type,
                                                                                  str(noise_ratio)))
        else:
            '''
            err_dir = os.path.join(err_base_dir, "%s_ratio%s" % (model_type,
                                                                 str(noise_ratio)))
            '''
            err_dir = os.path.join(err_base_dir, "%s" % model_type)
            
        for predict_domain in decoder_classes:
            augmented_errs[source_domain][target_domain][predict_domain] = HaoEvalDataset(os.path.join(err_dir,
                                                                                                       "predict_%s/errors.scp" % predict_domain))

print("Set up augmented dev datasets")

print("Done data setup")

Set up baseline dev datasets
Set up augmented dev datasets
Done data setup


In [4]:
# color_map = "coolwarm"
color_map = "viridis"
idx_to_domain = ["ihm", "sdm1"]
def plotParallelUttErrors(utt_id_idx):
    plt.clf()
    fig, axarr = plt.subplots(7, len(idx_to_domain), sharex=True)
    fig.set_size_inches(8.5, 11)
    
    for i in range(len(idx_to_domain)):
        predict_domain = idx_to_domain[i]
    
        # First show log mels for baseline
        utt_id = baseline_feats[predict_domain].utt_ids[utt_id_idx]
        feats = baseline_feats[predict_domain].feats_for_uttid(utt_id)
        axarr[0, i].axis('off')    # Pretty-up the resulting output by removing gridlines
        axarr[0, i].imshow(np.transpose(feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
        axarr[0, i].set_title("%s baseline feats" % predict_domain)
        
        # Show errors for baseline in- and out-domain
        row_idx = 1
        for source_domain in idx_to_domain:
            errs = baseline_errs[source_domain][predict_domain].feats_for_uttid(utt_id)
            axarr[row_idx, i].axis('off')    # Pretty-up the resulting output by removing gridlines
            axarr[row_idx, i].imshow(errs, origin='lower', cmap="RdYlGn", aspect='auto', interpolation='none')
            axarr[row_idx, i].set_title("%s baseline AM errors for %s" % (source_domain, predict_domain))
            row_idx += 1
            
        # Show errors for augmented data AMs
        for source_domain in idx_to_domain:
            for target_domain in idx_to_domain:
                '''
                adj_utt_id = "src_%s_tar_%s_%s" % (source_domain, target_domain, utt_id)
                errs = augmented_errs[source_domain][target_domain][predict_domain].feats_for_uttid(adj_utt_id)
                '''
                errs = augmented_errs[source_domain][target_domain][predict_domain].feats_for_uttid(utt_id)
                
                axarr[row_idx, i].axis('off')    # Pretty-up the resulting output by removing gridlines
                axarr[row_idx, i].imshow(errs, origin='lower', cmap="RdYlGn", aspect='auto', interpolation='none')
                axarr[row_idx, i].set_title("%s->%s baseline AM errors for %s" % (source_domain, target_domain, predict_domain))
                row_idx += 1
    
    plt.tight_layout()
    
    fig.savefig("%s_idx%d.eps" % (utt_id, utt_id_idx))
    
    plt.show()

# 43, 251 are good utterances (long + variations between IHM and SDM1)
interact(plotParallelUttErrors, utt_id_idx=range(len(baseline_feats["ihm"])))

A Jupyter Widget

<function __main__.plotParallelUttErrors>